In [1]:
import torch
from torch.autograd import Variable 
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [2]:
datasets

<module 'torchvision.datasets' from 'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\lib\\site-packages\\torchvision\\datasets\\__init__.py'>

In [4]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./mnist_data/
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [23]:
ls

 Volume in drive D is Local Disk
 Volume Serial Number is 927D-F5DC

 Directory of D:\spider_work

03/16/2019  11:12 AM    <DIR>          .
03/16/2019  11:12 AM    <DIR>          ..
03/16/2019  11:09 AM    <DIR>          .ipynb_checkpoints
03/15/2019  09:16 PM            35,788 Classify Code Base.ipynb
03/15/2019  09:26 PM            35,973 Classify Code Base-Copy1.ipynb
03/15/2019  09:27 PM            17,354 code_analysis.ipynb
03/15/2019  09:37 PM            19,297 code_analysis-Copy1.ipynb
03/15/2019  11:03 AM    <DIR>          data
03/14/2019  11:28 AM           103,557 dir_scan.txt
03/11/2019  02:50 PM    <DIR>          exp_spider_imo
10/31/2018  07:56 AM       428,028,722 exp_spider_imo.zip
03/14/2019  04:46 PM            38,676 file type classification.ipynb
03/16/2019  11:12 AM             3,708 Image Classification using CNN.ipynb
03/16/2019  11:11 AM            19,688 MNIST Pytorch.ipynb
03/15/2019  02:39 PM    <DIR>          mnist_data
03/13/2019  06:45 PM    <DIR>          

In [5]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())


In [21]:
train_dataset.targets[0]

tensor(5)

In [15]:
train_dataset.data.shape

torch.Size([60000, 28, 28])

In [6]:
class Net(torch.nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.L1 = torch.nn.Linear(28 * 28, 520)
#         self.L2 = torch.nn.Linear(520, 320)
#         self.L3 = torch.nn.Linear(320, 240)
#         self.L4 = torch.nn.Linear(240, 120)
#         self.L5 = torch.nn.Linear(120, 10)
        
#     def forward(self, x):
#         # reshape
#         x = x.view(-1, 28 * 28)
#         x = F.relu(self.L1(x))
#         x = F.relu(self.L2(x))
#         x = F.relu(self.L3(x))
#         x = F.relu(self.L4(x))
#         y_pred = self.L5(x)
#         return y_pred    
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()

In [ ]:
28 * 28

In [7]:
model.train()

Net(
  (l1): Linear(in_features=784, out_features=520, bias=True)
  (l2): Linear(in_features=520, out_features=320, bias=True)
  (l3): Linear(in_features=320, out_features=240, bias=True)
  (l4): Linear(in_features=240, out_features=120, bias=True)
  (l5): Linear(in_features=120, out_features=10, bias=True)
)

In [8]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [9]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
criterion

In [10]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
data

In [12]:

def train():
#     for idx, (data, target) in enumerate(train_loader):
#     #     print(data.shape)
#         x = Variable(data)
#         y = Variable(target)
#         y_pred = model(x)
#         optimizer.zero_grad()
#         #print(y_pred.shape, y.shape)

#         loss = criterion(y_pred, y)
#         if(idx % 60 == 0):
#             print(loss.item())

#         loss.backward()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        #print(output.shape, target.shape)
        loss = criterion(output, target)
#         break
        loss.backward()
        optimizer.step()
        if batch_idx % 60 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))
            


        
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        
        

for epoch in range(1, 10):
    train()
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.437357
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.408980
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.522663
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.416532
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.446243
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.241328
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.214510
Train Epoch: 1 [26880/60000 (45%)]	Loss: 0.181735
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.263920
Train Epoch: 1 [34560/60000 (58%)]	Loss: 0.338880
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.350732
Train Epoch: 1 [42240/60000 (70%)]	Loss: 0.313260
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.243402
Train Epoch: 1 [49920/60000 (83%)]	Loss: 0.325086
Train Epoch: 1 [53760/60000 (90%)]	Loss: 0.479992
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.235501


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0047, Accuracy: 9093/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.136563
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.264378
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.208050
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.488228
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.109092
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.443764
Train Epoch: 2 [23040/60000 (38%)]	Loss: 0.301298
Train Epoch: 2 [26880/60000 (45%)]	Loss: 0.193880
Train Epoch: 2 [30720/60000 (51%)]	Loss: 0.347144
Train Epoch: 2 [34560/60000 (58%)]	Loss: 0.199039
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.252732
Train Epoch: 2 [42240/60000 (70%)]	Loss: 0.442792
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.113935
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.342755
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.223267
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.176329

Test set: Average loss: 0.0034, Accuracy: 9361/10000 (93%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.159839
Train Epoch: 3 [3840/60000 (6%)]	Loss: 0.